# Emulating the LSST DRP Source Catalog with CosmoDC2Realizer

__Author:__ Ji Won Park (@jiwoncpark), __Last Run:__ 2018-12-14 (by @jiwoncpark)

__Goals:__
- In Part 1 (this notebook), learn how CosmoDC2Realizer emulates the LSST DRP Source Catalog of lensed quasars and contaminants from the CosmoDC2 extragalactic catalog and the truth catalog
- In Part 2, visualize sample light curves from the CosmoDC2Realized catalog

The following notebook was referenced to access and query the truth catalog:

    Scott Daniel's DC2 Tutorial truth_gcr_intro.ipynb

In [1]:
import os, sys
import numpy as np
from itertools import product
from scipy import spatial
from scipy.special import gammaincinv
import pandas as pd 
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import os, sys
sys.path.insert(0, '../utils')
import utils
# For reading in the OpSim database
import sqlite3
import healpy
# For accessing and querying the CosmoDC2 extragalactic catalog and truth catalog
#import GCRCatalogs
#from GCR import GCRQuery
%matplotlib inline
%load_ext autoreload
%autoreload 2

## About CosmoDC2Realizer

CosmoDC2Realizer is a framework that emulates the LSST DRP Source Catalog. It takes in two DC2 catalogs--the extragalactic and truth catalogs, which provide properties of extended galaxy sources and point sources (e.g. stars, AGNs), respectively--and the Opsim database, which provide the per-visit observation conditions. 

__Assumptions:__
- Emulation is made fast by bypassing image generation; we model each object as a mixture of Gaussians and the point-spread function (PSF) as a circular Gaussian so that we can _analytically_ compute the first and second moments required to populate the Source Catalog. 
- We also assume a fairly good deblender with a fixed deblending scale of 0.5"--chosen because it roughly corresponds to the full-width half maximum (FWHM) of the best LSST PSF. All sources located within the deblending scale of an object for a given visit will contribute to the moments of that object.

## 1. Choosing the OpSim fields
The OpSim database is organized in terms of 5292 viewing fields generated from a tesselation of the sky ([OpSim catalog schema documentation](https://www.lsst.org/scientists/simulations/opsim/summary-table-column-descriptions-v335)). The observing schedule and conditions are the same within each field so, for computational efficiency, CosmoDC2Realizer first identifies the set of fields over which to realize the comprising objects.

In [2]:
# Read in the minion_1016 opsim database
opsim_v3 = os.path.join('..', 'data', 'minion_1016_sqlite.db')
conn = sqlite3.connect(opsim_v3)

# See which tables the db file has
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[(u'Session',), (u'Config',), (u'Field',), (u'ObsHistory',), (u'Proposal',), (u'SeqHistory',), (u'SlewHistory',), (u'SlewActivities',), (u'SlewState',), (u'SlewMaxSpeeds',), (u'TimeHistory',), (u'ObsHistory_Proposal',), (u'Cloud',), (u'Seeing',), (u'Log',), (u'Config_File',), (u'Proposal_Field',), (u'SeqHistory_ObsHistory',), (u'MissedHistory',), (u'SeqHistory_MissedHistory',), (u'Summary',)]


We are primarily interested in two tables of the `minion_1016` database: `ObsHistory` containing the observation conditions and `Field` containing the field positions.

In [3]:
%%time # ~ 25s
# Save the tables ObsHistory and Field as Pandas DataFrames
obs_history = pd.read_sql(sql='SELECT * from ObsHistory', con=conn)
field = pd.read_sql(sql='SELECT * from Field', con=conn)

CPU times: user 24.1 s, sys: 2.2 s, total: 26.3 s
Wall time: 26.3 s


For speed considerations, we will only work with galaxies in `cosmoDC2_v1.0_9556`, a version of the extragalactic catalog restricted to one healpixel. This healpixel, it turns out, roughly coincides with the OpSim field with ID 1188 so we pre-save a subset of the `ObsHistory` table with `field_id=1188` and columns we'll need.

In [5]:
obs_history.columns

Index([u'obsHistID', u'Session_sessionID', u'filter', u'expDate', u'expMJD',
       u'night', u'visitTime', u'visitExpTime', u'finRank', u'finSeeing',
       u'transparency', u'airmass', u'vSkyBright', u'filtSkyBrightness',
       u'rotSkyPos', u'lst', u'altitude', u'azimuth', u'dist2Moon',
       u'solarElong', u'moonRA', u'moonDec', u'moonAlt', u'moonAZ',
       u'moonPhase', u'sunAlt', u'sunAZ', u'phaseAngle', u'rScatter',
       u'mieScatter', u'moonIllum', u'moonBright', u'darkBright', u'rawSeeing',
       u'wind', u'humidity', u'fiveSigmaDepth', u'ditheredRA', u'ditheredDec',
       u'Field_fieldID'],
      dtype='object')

In [ ]:
save_obs_history = os.path.join('..', 'data', 'obs_history_1188.csv')
save_field = os.path.join('..', 'data', 'field.csv')

obs_history_1188 = obs_history.loc[obs_history['Field_fieldID']==field_id]

obs_keep_cols = ['obsHistID', 'Field_fieldID', 'expMJD', 'finSeeing', 'fiveSigmaDepth', 'filtSkyBrightness', 'filter',]
obs_history_1188 = obs_history_1188[obs_keep_cols]

In [ ]:
'''
field.loc[(np.abs(field['fieldRA'] - 56.25) < 2) & (np.abs(field['fieldDec'] - (-34.24)) < 2)]
'''



In [ ]:
obs_history_1188.head()

In [ ]:
obs_history_1188.to_csv(save_obs_history, index=False)
field.to_csv(save_field, index=False)

From the `field` table, we can get the central RA and dec of our field. All fields have a 

## 2. Getting galaxies
We query the extragalactic catalog for objects that lie in this field. As mentioned earlier, we load `cosmoDC2_v1.0_9556` rather than the full cosmoDC2 catalog in this notebook for fast demonstration.

In [ ]:
%%time
catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.0_9556')
#catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.0_image')
# 'cosmoDC2_v1.0_image' takes ~14 sec
quantities = ['galaxy_id', 'ra_true', 'dec_true', 'redshift_true', 
              'size_bulge_true', 'size_minor_bulge_true', 'sersic_bulge', 'ellipticity_1_bulge_true',
              'ellipticity_2_bulge_true', 'ellipticity_bulge_true',
              'size_disk_true', 'size_minor_disk_true', 'sersic_disk', 'ellipticity_1_disk_true',
              'ellipticity_2_disk_true', 'ellipticity_disk_true',
              #'ellipticity_1_true', 'ellipticity_2_true',
              #'position_angle_true', 'ellipticity_true',
              #'size_true', 'size_minor_true', 'sersic',
              'bulge_to_total_ratio_i',
              'mag_true_u_lsst',
              'mag_true_g_lsst',
              'mag_true_r_lsst',
              'mag_true_i_lsst',
              'mag_true_z_lsst',
              'mag_true_Y_lsst',
              'is_central', 'halo_mass',]

cuts = [# A loose magnitude cut
        #GCRQuery('mag_true_g_lsst < 27'), 
        # Query halo masses likely to host an AGN
        GCRQuery('halo_mass > 1.e13'),
        # Query sources belonging to Field 1188
        GCRQuery('abs(ra_true - %f) < %f' %(field_ra, field_radius)),
        GCRQuery('abs(dec_true - %f) < %f' %(field_dec, field_radius)),]
# Add filters as necessary!
galaxies = catalog.get_quantities(quantities, filters=cuts)

We take a small subset of 1000 galaxies to realize. These galaxies will be at the center of our viewing window.

In [ ]:
small_galaxy_df.to_csv('small_galaxy_df.csv', index='galaxy_id')

## 3. Getting line-of-sight neighbors
For each extended galaxy source, any other galaxy or point source (unlensed AGN or star) that lie within its blending scale will be its line-of-sight neighbor. Galaxy neighbors will simply be taken from the extragalactic catalog, which we've already fetched. Point-source neighbors will be taken from the truth catalog as below.

### Getting the neighbors (unlensed AGNs and stars) from the truth catalog

In [ ]:
# truth_catalog = GCRCatalogs.load_catalog('dc2_truth_run1.1_static')
truth_catalog = GCRCatalogs.load_catalog('dc2_truth_run1.1', {'md5': None})

truth_catalog_columns = ['object_id', 'ra', 'dec', 'star', 'agn', 'sprinkled', 'healpix_2048',
                        'u', 'g', 'r', 'i', 'z', 'y',]

ra_min, ra_max = field_ra - field_radius, field_ra + field_radius
dec_min, dec_max = field_dec - field_radius, field_dec + field_radius

field_ra_rad = np.radians(field_ra)
field_dec_rad = np.radians(field_dec)

center_vec = np.array([np.cos(field_dec_rad)*np.cos(field_ra_rad),
                       np.cos(field_dec_rad)*np.sin(field_ra_rad),
                       np.sin(field_dec_rad)])

list_of_healpix = healpy.query_disc(2048, center_vec, np.radians(radius), nest=True, inclusive=True)

def filter_on_healpix(hp):
    return np.array([hh in list_of_healpix for hh in hp])

coord_filters = [
    'ra >= {}'.format(ra_min),
    'ra < {}'.format(ra_max),
    'dec >= {}'.format(dec_min),
    'dec < {}'.format(dec_max),
]

In [ ]:
%%time
# around 15s on Jupyter-dev
neighbors = truth_catalog.get_quantities(truth_catalog_columns,
                                            native_filters=['(star==1) or (agn==1)',
                                                            'sprinkled==0',
                                                            'healpix_2048<=%d' % list_of_healpix.max(),
                                                            'healpix_2048>=%d' % list_of_healpix.min()],
                                            filters=[(filter_on_healpix, 'healpix_2048')]+coord_filters)  
neighbors = pd.DataFrame(neighbors)

In [58]:
# delete later
data_dir = os.path.join('..', 'data')
obs_history = pd.read_csv(os.path.join(data_dir, 'obs_history_1188.csv'))
field = pd.read_csv(os.path.join(data_dir, 'field.csv'))
galaxies = pd.read_csv(os.path.join(data_dir, 'small_galaxy_df.csv'))
neighbors = pd.read_csv(os.path.join(data_dir, 'neighbors.csv'), index_col='object_id')
#TODO index=False when saving galaxy df

source_columns = ['sourceId', 'ccdVisitId', 'objectId',
                  'ra', 'dec', #'radec', #'radecCov',
                 'apFlux', 'apFluxErr',
                 'sky', #'skyErr',
                 'Ixx', 'Iyy', 'Ixy', #'Icov',
                 'IxxPSF', #'IyyPSF', 'IxyPSF',
                 #'extendedness',
                 ]
#source = pd.DataFrame(columns=source_columns)

deblending_scale = 0.5*100.0 # arcsec
# Information about the field we will work with
field_ids = [1188,]
field_radius = utils.deg_to_arcsec(0.5*3.5) # arcsec
field_id = 1188
#field_ra = 55.127186 #deg
#field_dec = -33.448002 #deg

# Pre-save tables field and obs_history as Pandas DataFrame
# Pre-save subset of extragalactic catalog corresponding to fields in field_ids as Pandas DataFrame
# Pre-save subset of truth catalog corresponding to fields in fields_ids as Pandas DataFrame

# TODO: make sure galaxies has galaxy_id
galaxies[['ra_true', 'dec_true']] = utils.deg_to_arcsec(galaxies[['ra_true', 'dec_true']])
neighbors[['ra', 'dec']] = utils.deg_to_arcsec(neighbors[['ra', 'dec']])
field[['fieldRA', 'fieldDec']] = utils.deg_to_arcsec(field[['fieldRA', 'fieldDec']])

moments = ['ra', 'dec', 'Ixx', 'Iyy', 'Ixy', 'apFlux']
filters = list('ugrizy')
moment_cols = [mom + '_' + bp for mom, bp in product(moments, filters)]
metadata_cols = ['sourceId', 'objectId', 'num_gal_neighbors', 'num_point_neighbors']

source_in_fields = []

for field_id in field_ids:
    # Query obs_history for the field of interest
    obs_history_in_field = obs_history.loc[obs_history['Field_fieldID']==field_id]
    num_observations = obs_history_in_field.shape[0]
    
    # Find field center for field id by querying the field table of OpSim db
    field_info = field.loc[field['fieldID']==field_id]
    field_ra = field_info['fieldRA'].item()
    field_dec = field_info['fieldDec'].item()
    
    # Query extragalactic catalog for galaxies within field
    gal_positions = np.c_[galaxies[['ra_true' ,'dec_true']].values]
    gal_tree = spatial.cKDTree(gal_positions)
    galaxies_in_field_idx = gal_tree.query_ball_point(x=[field_ra, field_dec], r=field_radius, p=2)
    galaxies_in_field = galaxies.iloc[galaxies_in_field_idx]
    num_galaxies = len(galaxies_in_field_idx)
    print(num_galaxies)
    
    # Query truth catalog for stars/AGNs within field
    point_positions = np.c_[neighbors[['ra', 'dec']].values]
    point_tree = spatial.cKDTree(point_positions)
    points_in_field_idx = point_tree.query_ball_point(x=[field_ra, field_dec], r=field_radius, p=2)
    points_in_field = neighbors.iloc[points_in_field_idx]
    
    # Initialize two DataFrames to populate before joining with obs_history_in_field
    moments_pre_observed = pd.DataFrame(columns=moment_cols)
    metadata_pre_observed = pd.DataFrame(columns=metadata_cols)
    
    for gal_idx in range(1): #range(num_galaxies):
        # Initialize dictionaries to be propagated for this blended system
        moments_row = {}
        metadata_row = {}
        
        # Central galaxy
        central_gal = galaxies_in_field.iloc[gal_idx]
        ra_center, dec_center = central_gal['ra_true'], central_gal['dec_true'] # pos of central galaxy
        metadata_row['sourceId'] = gal_idx
        metadata_row['objectId'] = central_gal['galaxy_id']
        
        ##########################
        # Find blended neighbors #
        ##########################
        # Galaxy neighbors (extended)
        gal_positions_field = np.c_[galaxies_in_field[['ra_true' ,'dec_true']].values]
        gal_tree_field = spatial.cKDTree(gal_positions_field)
        gal_idx = gal_tree_field.query_ball_point(x=[ra_center, dec_center], r=deblending_scale, p=2)
        num_gal_neighbors = len(gal_idx) - 1 # subtract central galaxy itself
        metadata_row['num_gal_neighbors'] = num_gal_neighbors
        all_gal = galaxies_in_field.iloc[gal_idx] # includes the central galaxy, not just neighbors
        # Stars/AGN neighbors (point)
        point_positions_field = np.c_[points_in_field[['ra', 'dec']].values]
        point_tree_field = spatial.cKDTree(point_positions_field)
        point_neighbors_idx = point_tree_field.query_ball_point(x=[ra_center, dec_center], r=deblending_scale, p=2)
        num_point_neighbors = len(point_neighbors_idx)
        metadata_row['num_point_neighbors'] = num_point_neighbors
        point = points_in_field.iloc[point_neighbors_idx]
        print(num_point_neighbors)
        
        # Separate galaxy catalog into bulge and disk
        bulge, disk, all_gal = separate_bulge_disk(all_gal)
        
        ###############
        # Flux ratios #
        ###############
        for bp in 'ugrizy':
            # Convert mag to flux in point_neighbors
            point['flux_%s' %bp] = utils.mag_to_flux(point[bp].values, to_unit='nMgy')
            # Store total flux of blended system
            moments_row['apFlux_%s' %bp] = all_gal['flux_%s' %bp].sum() +  point['flux_%s' %bp].sum()
            # Divide all fluxes by total blended flux, to get flux ratios
            total_blended_flux = moments_row['apFlux_%s' %bp] 
            all_gal['flux_%s' %bp] /= total_blended_flux
            bulge['flux_%s' %bp] /= total_blended_flux
            disk['flux_%s' %bp] /= total_blended_flux
            point['flux_%s' %bp] /= total_blended_flux
        
        ###############
        # 1st moments #
        ###############
        for bp in 'ugrizy':
            # Calculate contribution to 1st moment in each band and gal/point of blended system
            all_gal['Ix_contrib_%s' %bp] = all_gal['ra_true']*all_gal['flux_%s' %bp]
            all_gal['Iy_contrib_%s' %bp] = all_gal['dec_true']*all_gal['flux_%s' %bp]
            point['Ix_contrib_%s' %bp] = point['ra']*point['flux_%s' %bp]
            point['Iy_contrib_%s' %bp] = point['dec']*point['flux_%s' %bp]
            # Sum to get 1st moment of blended system
            moments_row['ra_%s' %bp] = all_gal['Ix_contrib_%s' %bp].sum() + point['Ix_contrib_%s' %bp].sum()
            moments_row['dec_%s' %bp] = all_gal['Iy_contrib_%s' %bp].sum() + point['Iy_contrib_%s' %bp].sum()
        
        ###############
        # 2nd moments #
        ###############
        # Deconstruct bulge/disk into MoG
        bulge_mog = sersic_to_mog(sersic_df=bulge, bulge_or_disk='bulge')
        disk_mog = sersic_to_mog(sersic_df=disk, bulge_or_disk='disk')
        # Calculate contribution to 1st moment
        # ... in each band and Gaussian of blended system
        bulge_mog = calculate_2nd_moment_contrib_mog(mog_df=bulge_mog, moments_dict=moments_row)
        disk_mog = calculate_2nd_moment_contrib_mog(mog_df=disk_mog, moments_dict=moments_row)
        # ... in each band and point of blended system
        point = calculate_2nd_moment_contrib_point(point_df=point, moments_dict=moments_row)
        # Sum to get 2nd moment of blended system
        for bp in 'ugrizy':
            for mom in ['Ixx', 'Iyy', 'Ixy']:
                final_mom = bulge_mog['%s_contrib_%s' %(mom, bp)].sum() 
                final_mom += disk_mog['%s_contrib_%s' %(mom, bp)].sum() 
                final_mom += point['%s_contrib_%s' %(mom, bp)].sum() 
                moments_row['%s_%s' %(mom, bp)] = final_mom
    
        moments_pre_observed = moments_pre_observed.append(moments_row, ignore_index=True)
        metadata_pre_observed = metadata_pre_observed.append(metadata_row, ignore_index=True)
        

        

504
0


/home/jwp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/jwp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/jwp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [54]:
bulge_mog.columns

Index([u'index', u'size_minor_true', u'bulge_to_total_ratio_i',
       u'ellipticity_1_true', u'ellipticity_2_true', u'ellipticity_true',
       u'size_true', u'sersic', u'flux_u', u'flux_g', u'flux_r', u'flux_i',
       u'flux_z', u'flux_y', u'ra_true', u'dec_true', u'size_circular',
       u'gauss_norm', u'stdev', u'weight', u'Ixx_contrib_u', u'Iyy_contrib_u',
       u'Ixy_contrib_u', u'Ixx_contrib_g', u'Iyy_contrib_g', u'Ixy_contrib_g',
       u'Ixx_contrib_r', u'Iyy_contrib_r', u'Ixy_contrib_r', u'Ixx_contrib_i',
       u'Iyy_contrib_i', u'Ixy_contrib_i', u'Ixx_contrib_z', u'Iyy_contrib_z',
       u'Ixy_contrib_z', u'Ixx_contrib_y', u'Iyy_contrib_y', u'Ixy_contrib_y'],
      dtype='object')

In [53]:
moments_pre_observed

,ra_u,ra_g,ra_r,ra_i,ra_z,ra_y,dec_u,dec_g,dec_r,dec_i,dec_z,dec_y,Ixx_u,Ixx_g,Ixx_r,Ixx_i,Ixx_z,Ixx_y,Iyy_u,Iyy_g,Iyy_r,Iyy_i,Iyy_z,Iyy_y,Ixy_u,Ixy_g,Ixy_r,Ixy_i,Ixy_z,Ixy_y,apFlux_u,apFlux_g,apFlux_r,apFlux_i,apFlux_z,apFlux_y
0,200482.468863,200482.478684,200482.48692,200482.491487,200482.491441,200482.493494,-126294.671992,-126294.638848,-126294.610891,-126294.596126,-126294.595327,-126294.587597,2.682322,2.486058,2.321107,2.230513,2.230216,2.188078,11.316441,10.320565,9.4797,9.031149,9.012038,8.782953,4.171484,3.827102,3.537695,3.377421,3.378493,3.305762,1.969478,9.309461,25.257668,38.895389,48.970699,58.498118


In [ ]:
pd.concat([])

In [ ]:
moments = ['totalflux', 'Ix', 'Ixx', 'Iy', 'Iyy', 'Ixy']


In [39]:
def separate_bulge_disk(extragal_df):
    # Rename for conciseness
    df = extragal_df
    # Add disk_to_total_ratio_i = 1.0 - bulge_to_total_ratio_i
    df['disk_to_total_ratio'] = 1.0 - df['bulge_to_total_ratio_i'].values
    # Make column names all lowercase for convenience
    df.columns = map(str.lower, df.columns)
    # Add flux column for each filter, disk/bulge flux
    for bandpass in 'ugrizy':
        df['flux_%s' %bandpass] = utils.mag_to_flux(df['mag_true_%s_lsst' %bandpass].values, to_unit='nMgy')
        df['flux_%s_disk' %bandpass] = df['flux_%s' %bandpass].values*df['disk_to_total_ratio'].values
        df['flux_%s_bulge' %bandpass] = df['flux_%s' %bandpass].values*df['bulge_to_total_ratio_i'].values
    # Copy ra and dec to both disk and bulge DataFrames (not sure if necessary)
    for component in ['disk', 'bulge']:
        df['ra_true_%s' %component] = df['ra_true'].values
        df['dec_true_%s' %component] = df['dec_true'].values
    # Separate df into bulge-related and disk-related
    bulge_df = df.filter(like='bulge', axis=1).copy()
    disk_df = df.filter(like='disk', axis=1).copy()
    # Make column schema the same across bulge and disk DataFrames (not sure if necessary)
    bulge_df.columns = [col.strip().replace('_bulge', '') for col in bulge_df.columns]
    disk_df.columns = [col.strip().replace('_disk', '') for col in disk_df.columns]
    # Sersic size before shear transformation
    bulge_df['size_circular'] = (bulge_df['size_minor_true']*bulge_df['size_true'])**0.5
    disk_df['size_circular'] = (disk_df['size_minor_true']*disk_df['size_true'])**0.5
    return bulge_df, disk_df, df

In [49]:
def sersic_to_mog(sersic_df, bulge_or_disk):
    
    if bulge_or_disk=='bulge':
        # Mixture of gaussian parameters for de Vaucouleurs profile from HL13
        weights = [0.00139, 0.00941, 0.04441, 0.16162, 0.48121, 1.20357, 2.54182, 4.46441, 6.22821, 6.15393]
        stdevs = [0.00087, 0.00296, 0.00792, 0.01902, 0.04289, 0.09351, 0.20168, 0.44126, 1.01833, 2.74555]
        mog_params = {'weight': weights, 'stdev': stdevs}
        sersic_norm = gammaincinv(8, 0.5)
        gauss_norm = 40320.0*np.pi*np.exp(sersic_norm)/sersic_norm**8.0
    elif bulge_or_disk=='disk':
        # Mixture of gaussian parameters for exponential profile from HL13
        weights = [0.00077, 0.01017, 0.07313, 0.37188, 1.39727, 3.56054, 4.74340, 1.78732]
        stdevs = [0.02393, 0.06490, 0.13580, 0.25096, 0.42942, 0.69672, 1.08879, 1.67294]
        mog_params = {'weight': weights, 'stdev': stdevs}
        sersic_norm = gammaincinv(2, 0.5) # for exponential
        gauss_norm = 2.0*np.pi*np.exp(sersic_norm)/sersic_norm**2.0
    else:
        raise ValueError("Component is either bulge or disk.")
    
    sersic_df['gauss_norm'] = gauss_norm
    mog_params_df = pd.DataFrame.from_dict(mog_params)
    # Join bulge_df and mog_params_df
    sersic_df = sersic_df.reset_index()
    sersic_df['key'] = 0
    mog_params_df['key'] = 0
    mog = sersic_df.merge(mog_params_df, how='left', on='key')
    mog = mog.drop('key', 1)
    
    return mog

In [48]:
def calculate_2nd_moment_contrib_mog(mog_df, moments_dict):

    # Calculate contribution to 2nd moment from each filter and Gaussian
    for bp in 'ugrizy':
        e1 = mog_df['ellipticity_1_true']
        e2 = mog_df['ellipticity_2_true']
        gauss_sigma = mog_df['size_circular']*mog_df['stdev']
        gauss_weight = mog_df['weight']/mog_df['gauss_norm']
        flux_ratio = mog_df['flux_%s' %bp]*gauss_weight
        ra = mog_df['ra_true']
        dec = mog_df['dec_true']
        reference_Ix = moments_dict['ra_%s' %bp]
        reference_Iy = moments_dict['dec_%s' %bp]
        Ixx, Iyy, Ixy = get_second_moments(e1=e1, e2=e2, sigma=gauss_sigma)
        mog_df['Ixx_contrib_%s' %bp] = flux_ratio*(Ixx + (ra - reference_Ix)**2.0)
        mog_df['Iyy_contrib_%s' %bp] = flux_ratio*(Iyy + (dec - reference_Iy)**2.0)
        mog_df['Ixy_contrib_%s' %bp] = flux_ratio*(Ixy + (ra - reference_Ix)*(dec - reference_Iy))
    
    return mog_df

In [51]:
def calculate_2nd_moment_contrib_point(point_df, moments_dict):

    # Calculate contribution to 2nd moment from each filter and Gaussian
    for bp in 'ugrizy':
        flux_ratio = point_df['flux_%s' %bp]
        ra = point_df['ra']
        dec = point_df['dec']
        reference_Ix = moments_dict['ra_%s' %bp]
        reference_Iy = moments_dict['dec_%s' %bp]
        point_df['Ixx_contrib_%s' %bp] = flux_ratio*((ra - reference_Ix)**2.0)
        point_df['Iyy_contrib_%s' %bp] = flux_ratio*((dec - reference_Iy)**2.0)
        point_df['Ixy_contrib_%s' %bp] = flux_ratio*((ra - reference_Ix)*(dec - reference_Iy))
    
    return point_df

In [43]:
def get_second_moments(e1, e2, sigma):
    theta = 0.5*np.arctan(e2/e1)
    e = (e1**2.0 + e2**2.0)**0.5
    sqrt_q = ((1.0 - e)/(1.0 + e))**0.5
    lam1 = sigma**2.0/sqrt_q
    lam2 = sigma**2.0*sqrt_q
    cos = np.cos(theta)
    sin = np.sin(theta)
    Ixx = lam1*cos**2.0 + lam2*sin**2.0
    Iyy = lam1*sin**2.0 + lam2*cos**2.0
    Ixy = (lam1 - lam2)*cos*sin
    return Ixx, Iyy, Ixy

In [57]:
np.arctan(0.0/0.0)


ZeroDivisionError: float division by zero

In [55]:

print(b4, b1)

for bp in 'ugrizy':
    e1 = obj['ellipticity_1_true']
    e2 = obj['ellipticity_2_true']
    gauss_sigma = obj['size_circular']*obj['stdev']
    gauss_weight = obj['weight']/gauss_norm
    flux_ratio = obj['flux_%s' %bp]*gauss_weight
    ra = obj['ra_true']
    dec = obj['dec_true']
    Ix = derived_properties['Ix_%s' %bp]
    Iy = derived_properties['Iy_%s' %bp]
    Ixx, Iyy, Ixy = get_second_moments(e1=e1, e2=e2, sigma=gauss_sigma)
    obj['Ixx_contrib_%s' %bp] = flux_ratio*(Ixx + (ra - Ix)**2.0)
    obj['Iyy_contrib_%s' %bp] = flux_ratio*(Iyy + (dec - Iy)**2.0)
    obj['Ixy_contrib_%s' %bp] = flux_ratio*(Ixy + (ra - Ix)*(dec - Iy))
    derived_properties['Ixx_%s' %bp] = obj['Ixx_contrib_%s' %bp].sum()
    derived_properties['Iyy_%s' %bp] = obj['Iyy_contrib_%s' %bp].sum()
    derived_properties['Ixy_%s' %bp] = obj['Ixy_contrib_%s' %bp].sum()
    

NameError: name 'b4' is not defined

In [ ]:
meta = {'a': 1, 'b': 2, 'c': 3}
#mom = {'d': 4, 'e': 5}


In [ ]:
derived_properties

In [ ]:
before_observed = pd.DataFrame(columns=derived_properties.keys())

before_observed = before_observed.append(derived_properties, ignore_index=True)

In [ ]:
#before_observed.columns = before_observed.columns.str.split('_', expand=True)

In [ ]:
#before_observed = before_observed.reorder_levels([1,0], axis=1)

In [ ]:
#before_observed = before_observed.sort_index(axis=1)

In [ ]:
before_observed

In [ ]:
obs_history_in_field = obs_history.loc[obs_history['Field_fieldID']==field_id][['filter']]
#obs_history_in_field.columns = pd.MultiIndex.from_product([['obs'], obs_history_in_field.columns])
obs_history_in_field = obs_history_in_field.sort_index(axis=1)

In [ ]:
before_observed['key'] = 0
obs_history_in_field['key'] = 0
obj = before_observed.merge(obs_history_in_field, how='left', on='key')
obj = obj.drop('key', 1)

In [ ]:
obj.head()

In [ ]:
obj.sort_index(axis=1)